In [91]:
# Project olive tree
# let's grow olive treeeees!

import re
import pandas as pd
import requests
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
import os 
#from config import Config
#import openpyxl as op 

In [92]:
url = 'https://www.oliveyoung.co.kr/store/main/main.do?oy=0' 
r = requests.get(url)

if r.status_code == 200:
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')

else : 
    print(r.status_code)

In [93]:
events = soup.find_all(class_ = 'banner_link')

banner_title = []
banner_desc = []
banner_link = []

for event in events : 

    
    try : 
        if 'PlanShop' in event["href"]  : 
            banner_title.append(event.find("span").text)
            banner_desc.append(event.text)
            banner_link.append(event["href"])
        else : 
            continue
    except : 
        continue

In [94]:
df1 = pd.DataFrame({"banner_title" : banner_title, 
            "banner_desc" : banner_desc,
            "banner_link" : banner_link} 
             )

df1.head()

,banner_title,banner_desc,banner_link
0,올영세일,\n\n올영세일\n감탄을 부르는올마이갓올영세일\n최대 70%할인 누려요\n\n\n,https://www.oliveyoung.co.kr/store/planshop/ge...
1,올영세일\r,\n\n올영세일\r\n싱그러운\r봄 컬러를\r담아요\r\n색조 BEST 최대 44%...,https://www.oliveyoung.co.kr/store/planshop/ge...
2,올영세일,\n\n올영세일\n봄철미세먼지와햇살을 이겨요\n클렌징/썬 최대 53%할인\n\n\n,https://www.oliveyoung.co.kr/store/planshop/ge...
3,올영세일,\n\n올영세일\n삶의질상승을 위한아이템\n건강/위생 최대 59%할인\n\n\n,https://www.oliveyoung.co.kr/store/planshop/ge...
4,올영세일,\n\n올영세일\n환절기 속건강 루틴만들기\n건강/일반식품 최대 55%할인\n\n\n,https://www.oliveyoung.co.kr/store/planshop/ge...


In [99]:
promo_title = []
promo_start_dt = []
promo_end_dt = []
promo_contents = []
promo_catgs = []

for l in banner_link : 
    url = l
    r = requests.get(url)

    if r.status_code == 200:
        html = r.text
        soup = BeautifulSoup(html, 'html.parser')

    else : 
        print(r.status_code)

    #title 
    try : 
        title = soup.find(id = "planTitle").text
    except : 
        title = 'none(기타)'
    #period
    try : 
        period = soup.find(class_ = "title-plan-view").find("span").text
        period = period.replace("\n", "").replace("\t", "").replace(" ", "")
        start_dt = period.split("-")[0]
        end_dt = period.split("-")[1]
    except : 
        start_dt = 'none'
        end_dt = 'none'
    #desc
    try : 
        desc = soup.find_all(class_ = 'oyblind')
        desc = [d.text.replace("\n", " ").replace("  ", " ").replace("  ", " ") for d in desc]
    except : 
        desc = 'none'
    #catg
    try : 
        catg = soup.find_all(class_ = 'plan-menu')[0].text.split("\n")
        catg = [c for c  in catg if c != '']
    except :
        catg = 'none'

    promo_title.append(title)
    promo_start_dt.append(start_dt)
    promo_end_dt.append(end_dt)
    promo_contents.append(desc)
    promo_catgs.append(catg)

    #img save
    promo_imgs = soup.find_all("img")
    promo_imgs = [i["src"] for i in promo_imgs if (("goods" not in i["src"]) and ('https' not in i["src"])) or ("planshop" in i["src"])]
    promo_imgs = ['https:'+i if "http" not in i else i for i in promo_imgs]
    path = "./result/images/" + start_dt.replace(".","") + title
    try : 
        os.mkdir(path)
        
        for num, img in enumerate(promo_imgs) : 
            urllib.request.urlretrieve(img, path + f"/{num}.jpg")
    except : 
        continue 



In [96]:
df2 = pd.DataFrame({"banner_link" : banner_link, 
            "promo_title" : promo_title, 
            "promo_start_dt" : promo_start_dt,
            "promo_end_dt" : promo_end_dt, 
            "promo_contents" : promo_contents, 
            "promo_catgs" : promo_catgs} 
             )

In [97]:
result = pd.merge(left= df1, right = df2, how= "left")
result = result[["promo_title", "promo_start_dt", "promo_end_dt", "banner_title", "banner_desc", "promo_contents", "promo_catgs", "banner_link"]]
result.head()

,promo_title,promo_start_dt,promo_end_dt,banner_title,banner_desc,promo_contents,promo_catgs,banner_link
0,올마이갓 올영세일🌹,23.03.02,23.03.08,올영세일,\n\n올영세일\n감탄을 부르는올마이갓올영세일\n최대 70%할인 누려요\n\n\n,"[ 올영세일 3/2~3/8 , 다양한 혜택이 있는 올리브영 APP에서올영세일을 즐...","[전체, 프리미엄💄, 올영세일 추천템👍, 인기테마특가⭐]",https://www.oliveyoung.co.kr/store/planshop/ge...
1,봄 메이크업 필수템 NEW&BEST🌺,23.03.02,23.03.08,올영세일\r,\n\n올영세일\r\n싱그러운\r봄 컬러를\r담아요\r\n색조 BEST 최대 44%...,[ 봄 취향저격 메이크업 BEST & TREND ],"[전체, 🌼3/2 오늘의 특가🌼, 베이스 NEW&BEST, 아이 NEW&BEST, ...",https://www.oliveyoung.co.kr/store/planshop/ge...
2,강력세정💧 강력차단⚔️,23.03.02,23.03.08,올영세일,\n\n올영세일\n봄철미세먼지와햇살을 이겨요\n클렌징/썬 최대 53%할인\n\n\n,[ 봄 맞이 강력세정 & 강력차단 클렌징/썬 BEST & TREND ],"[전체, 3/2 오늘의 특가, 단 7일! 단독특가💥, 강력세정 클렌징 최대특가💦, ...",https://www.oliveyoung.co.kr/store/planshop/ge...
3,⭐건강위생용품 BEST 특가⭐,23.03.02,23.03.08,올영세일,\n\n올영세일\n삶의질상승을 위한아이템\n건강/위생 최대 59%할인\n\n\n,[ 행복 TO THE MAX 꼭 사야 할 데일리 필수템 ],"[전체, ✔3/2 오늘의 특가, 생리용품 BEST, 구강용품 BEST, 청결제 BE...",https://www.oliveyoung.co.kr/store/planshop/ge...
4,🌼맛있게 챙기는 봄 건강루틴🌸,23.03.02,23.03.08,올영세일,\n\n올영세일\n환절기 속건강 루틴만들기\n건강/일반식품 최대 55%할인\n\n\n,[ 맛있게 챙기는 봄 건강 루틴 만들기 ],"[전체, 🌼3/2 하루 특가🌼, 단 7일, FOOD 특가, 건강식품 7일 특가, 슬...",https://www.oliveyoung.co.kr/store/planshop/ge...


In [98]:
update_dt = datetime.now()
result_path = f'./result/result_{update_dt}.csv'

result.to_csv(result_path, encoding = 'utf-8-sig', index = False)